In [1]:
import pandas as pd
from neuralforecast.core import NeuralForecast
from neuralforecast.models import TimesNet
import torch

/home/maybe/miniconda3/envs/task2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-16 14:40:41,720	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-16 14:40:41,837	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# 2. Гиперпараметры
horizon = 40
input_size = 200

In [3]:
# 1. Загрузка и подготовка данных
df = pd.read_csv('evs1_012_10_bar_harmonics_median_new.csv', parse_dates=['date'])
df = df.rename(columns={'date': 'ds', 'value': 'y'})
df['unique_id'] = 'id_1'

In [4]:
# 3. Инициализация и обучение модели
model = TimesNet(
    h=horizon,
    input_size=input_size,
    max_steps=60,
    learning_rate=1e-3,
    scaler_type='standard'
)
nf = NeuralForecast(models=[model], freq='D')
nf.fit(df=df)

Seed set to 1
/home/maybe/miniconda3/envs/task2/lib/python3.10/site-packages/neuralforecast/common/_base_model.py:537: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type          | Params | Mode 
---------------------------------------------------------
0 | loss           | MAE           | 0      | train
1 | padder_train   | ConstantPad1d | 0      | train
2 | scaler         | T

Epoch 59: 100%|██████████| 1/1 [00:00<00:00,  2.37it/s, v_num=0, train_loss_step=0.265, train_loss_epoch=0.265]

`Trainer.fit` stopped: `max_steps=60` reached.


Epoch 59: 100%|██████████| 1/1 [00:00<00:00,  2.36it/s, v_num=0, train_loss_step=0.265, train_loss_epoch=0.265]


In [5]:
# 4. Сохранение модели
# Сохраняет state_dict TimesNet и объект scaler, если требуется
torch.save(model.model.state_dict(), 'timesnet_state_dict.pth')
nf.save('neuralforecast_nf.pkl')  # Сохраняет весь контейнер (если потребуется)